In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta 

**Read CSV**

In [3]:
# from google.colab import files
# uploaded = files.upload()
# import pandas as pd
# import io

# bike = pd.read_csv(io.StringIO(uploaded['201801_capitalbikeshare_tripdata.csv'].decode('utf-8')))
file = '../1-raw_data/2018/201801_capitalbikeshare_tripdata.csv'
bike = pd.read_csv(file, sep = ",")

In [ ]:
Start_hour = []
End_hour = []
v = bike['Start date'].str.split()
w = bike['End date'].str.split()
for i in range (len(bike['Start date'])):
    Start_hour.append(v[i][1].split(":")[0])
    End_hour.append(w[i][1].split(":")[0])
bike["Start hour"] = Start_hour
bike["End hour"] = End_hour

**Filter out trips that start and end at the same station and the trip duration is below the threshold we set**

In [ ]:
bike1 = bike.copy()
bike1 = bike1[bike1['Start station number'] != bike1['End station number']]
len(bike1)

# same_station = bike[bike['Start station number'] == bike['End station number']]
# # len(same_station)
# len(same_station[same_station['Duration'] <300])

**Filter out the station has only a few pickups or returns**

In [ ]:
filtered1 = bike1.groupby('Start station').filter(lambda g: g.Duration.count() > 45)
len(filtered1)
#.Duration.mean()
grouped2 = bike.groupby("End station").count()
grouped2[grouped2.Duration < 45]
grouped2['Duration'].describe()
# bottom 25% is 45. 
filtered = bike1.groupby('Start station').filter(lambda x: x.Duration.count() > 45)
len(filtered)
filtered_data = filtered[filtered['Member type'] == 'Member']
len(filtered_data)

In [ ]:
# max_d = bike['Duration'].max()
# min_d = bike['Duration'].min()
# bike.Duration = bike.Duration.apply(lambda x: (1.0*x - min_d)/(max_d-min_d))

In [ ]:
# bs_31321 = filtered_data[filtered_data["Start station number"] == 31321]
# week_count = [0]*7
# for i in range(1, 32):
#     if i < 9:
#         temp1 = bs_31321[(bs_31321["Start date"] >= ("2018-01-0" + str(i))) & (bs_31321["Start date"] < ("2018-01-0" + str(i+1)))]
#         week_count[np.mod(i,7)-1] = week_count[np.mod(i,7)-1]  + len(temp1)
#     elif i == 9:
#         temp2 = bs_31321[(bs_31321["Start date"] >= ("2018-01-0" + str(i))) & (bs_31321["Start date"] < ("2018-01-" + str(i+1)))]
#         week_count[np.mod(i,7)-1] = week_count[np.mod(i,7)-1]  + len(temp2)
#     else:
#         temp3 = bs_31321[(bs_31321["Start date"] >= ("2018-01-" + str(i))) & (bs_31321["Start date"] < ("2018-01-" + str(i+1)))]
#         week_count[np.mod(i,7)-1] = week_count[np.mod(i,7)-1]  + len(temp3)
#     i = i + 1
# sum(week_count)

In [ ]:
def bike_pick_up(station):
    bike_station = filtered_data[filtered_data["Start station number"] == station]
    pick_up = [0]*24
    hour = list(bike_station['Start hour'])
    for i in range(len(hour)):
        pick_up[int(hour[i])] = pick_up[int(hour[i])] + 1
    if sum(pick_up) is not 0:    
        pick_up= [float(item) / sum(pick_up) for item in pick_up]
    return pick_up

In [ ]:
def bike_return(station):
    bike_station = filtered_data[filtered_data["End station number"] == station]
    b_return = [0]*24
    hour = list(bike_station['End hour'])
    for i in range(len(hour)):
        b_return[int(hour[i])] = b_return[int(hour[i])] + 1
    if sum(b_return) is not 0:    
        b_return = [float(item) / sum(b_return) for item in b_return]
    return b_return

In [ ]:
transferred = pd.DataFrame(columns=['Station', 'p0-1', 'p1-2','p2-3','p3-4','p4-5','p5-6','p6-7','p7-8','p8-9','p9-10','p10-11',
                                    'p11-12','p12-13','p13-14','p14-15','p15-16','p16-17','p17-18','p18-19','p19-20','p20-21',
                                    'p21-22','p22-23','p23-0','r0-1', 'r1-2','r2-3','r3-4','r4-5','r5-6','r6-7','r7-8','r8-9',
                                    'r9-10','r10-11','r11-12','r12-13','r13-14','r14-15','r15-16','r16-17','r17-18','r18-19',
                                    'r19-20','r20-21','r21-22','r22-23','r23-0'])
i = 0
bike_station = set(filtered_data["Start station number"])
for station in bike_station:
    b_station = [str(station)]
    b_pickup = bike_pick_up(station)
    b_return = bike_return(station) 
    row = b_station + b_pickup+b_return
    transferred.loc[i] = row
    i = i+1
transferred

# Everything using dataset filtering start_station != end_station, count > 45, member type = member 



In [ ]:
transferred.to_csv(r'datacleaning.csv')